In [ ]:
from sec_edgar_downloader import Downloader
import os

# Initialize the downloader
dl = Downloader("YourCompanyName", "your.email@domain.com")

# Download all 10-Q filings for Microsoft
dl.get("10-Q", "MSFT")

# Define the directory where filings are downloaded
# Adjust the path to match the location where filings are saved
directory = "sec-edgar-filings/MSFT/10-Q/"

# Print the file names of all downloaded 10-Q filings for Microsoft
if os.path.exists(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".txt") or filename.endswith(".xml"):  # Adjust based on file formats
            print(filename)
else:
    print(f"Directory {directory} does not exist.")


In [ ]:
import requests
import json
import logging

def fetch_microsoft_filings(file_path="microsoft_filings.json"):
    # Define the URL for Microsoft's filings data
    url = "https://data.sec.gov/submissions/CIK0000789019.json"
    
    # Define headers to include with the request
    headers = {
        'User-Agent': 'Sample Company Name AdminContact@<sample company domain>.com'
    }
    
    try:
        # Send a GET request to the URL with headers
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        response.raise_for_status()
        
        # Load the JSON data from the response
        data = response.json()
        
        # Save the JSON data to a file
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
        
        logging.info(f"Data successfully saved to {file_path}")
    
    except requests.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except Exception as err:
        logging.error(f"An error occurred: {err}")

In [ ]:
def load_json_as_dict(file_path="microsoft_filings.json"):
    try:
        # Open and load the JSON data from the file
        with open(file_path, 'r') as file:
            data = json.load(file)
        logging.info(f"Data successfully loaded from {file_path}")
        return data
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
    except json.JSONDecodeError as json_err:
        logging.error(f"Error decoding JSON: {json_err}")
    except Exception as err:
        logging.error(f"An error occurred: {err}")

data = load_json_as_dict()

In [ ]:
def get_accession_numbers(data):
    try:
        # Access the nested dictionary keys directly
        filings = data["filings"]
        print(filings)
        recent = filings["recent"]
        accession_numbers = recent["accessionNumber"]
        if not accession_numbers:
            logging.warning("No accession numbers found.")
        return accession_numbers
    except KeyError as key_err:
        logging.error(f"Key error: {key_err}")
        return []
    except Exception as err:
        logging.error(f"An error occurred while extracting accession numbers: {err}")
        return []


def fetch_sec_filing(accession_number):
    # Base URL for SEC filings
    base_url = "https://www.sec.gov/Archives/edgar/data/"
    
    # Central Index Key (CIK) for Microsoft
    cik = "0000789019"
    
    # Construct the URL for the filing
    url = f"{base_url}{cik}/{accession_number}/index.json"
    
    # Define headers to include with the request
    headers = {
        'User-Agent': 'Sample Company Name AdminContact@<sample company domain>.com'
    }
    
    try:
        # Send a GET request to the URL with headers
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        response.raise_for_status()
        
        # Load the JSON data from the response
        data = response.json()
        
        return data
    except requests.HTTPError as http_err:
        logging.error(f"HTTP error occurred while fetching {accession_number}: {http_err}")
    except Exception as err:
        logging.error(f"An error occurred while fetching {accession_number}: {err}")
    
    return None

def fetch_latest_sec_data(accession_numbers):
    latest_data = {}
    
    for number in accession_numbers:
        # Fetch the SEC filing data for each accession number
        data = fetch_sec_filing(number)
        if data:
            latest_data[number] = data
    
    return latest_data
accession_numbers = get_accession_numbers(data)
fetch_latest_sec_data(accession_numbers=accession_numbers)
